In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install MTCNN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.1 MB/s eta 0:00:00


In [ ]:
import cv2
import requests
from io import BytesIO
# extract and plot each detected face in a photograph
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn import MTCNN
# from google.colab.patches import cv2_imshow
import zipfile
import os
import numpy as np

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image

##https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi

In [ ]:

class Data_Using:


  def unzip(self,zip_dir,out_dir):
    path_folder = out_dir

    zip_file_path = f'{zip_dir}'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(path_folder)
  #  if not(os.path.exists(self.path_folder)):
  #  else :
      # print("Files Exists")


  def load_data_image(self,path_image):
    img = cv2.imread(path_image)
    return img


  def load_data_video(self,path_video):

    frame_list = []
    # Load the video content using OpenCV
    cap = cv2.VideoCapture(path_video)
    # Loop through the frames and display them
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Display the frame
        # cv2_imshow( frame)

        frame_list.append(frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()
    frame_list = np.array(frame_list)
    return frame_list


  def take_photo(self,filename='photo.jpg', quality=0.8):
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
      }
      ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
    return filename


  def load_data_colab_camera(self):
    try:
      filename = self.take_photo()
      print('Saved to {}'.format(filename))

      # Show the image which was just taken.
      display(Image(filename))
      # if cv2.waitKey(1) & 0xFF == ord('q'):
      #   cv2.destroyAllWindows()
    except Exception as err:
      # Errors will be thrown if the user does not have a webcam or if they do not
      # grant the page permission to access it.
      print(str(err))


  def load_data_system_camera(self):
    # define a video capture object
    vid = cv2.VideoCapture(0)
    while(True):
        # Capture the video frame
        ret, frame = vid.read()
        # Display the resulting frame
        cv2.imshow('frame', frame)
        cv2.imwrite('video.mp4',frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()



In [26]:
from genericpath import exists
# Face detection from video dataset

class Face_Detection_Recognition_Video:
    def __init__(self):
      # super.__init__(self)
      self.face_number = 0
      self.folder_number = 0

    def preprocess(self,image):
      #normalize
        pixels = image.astype('float32')
        pixels /= 255.0
        #resize
        # img_normalized = cv2.resize(pixels,(1024,1024))
        return pixels

      # draw each face separately
    def mtcnn_model(self,filename, result_list):
        # load the image
        # data = pyplot.imread(filename)
        data = filename

        # pyplot.imshow(data)
        # preprocess on image
        # data = preprocess(data)
        # get the context for drawing boxes
        ax = pyplot.gca()
        for result in result_list:
          # get coordinates
          x, y, width, height = result['box']
          # create the shape
          rect = Rectangle((x, y), width, height, fill=False, color='red')
          # draw the box
          ax.add_patch(rect)
        # pyplot.show()

        # draw the dots
        for key, value in result['keypoints'].items():
          # create and draw dot
          dot = Circle(value, radius=2, color='red')
          ax.add_patch(dot)
        # show the plot
        # pyplot.show()
        crop_face = []
        for i in range(len(result_list)):
          # get coordinates
          x1, y1, width, height = result_list[i]['box']
          x2, y2 = x1 + width, y1 + height
          # define subplot
          pyplot.subplot(1, len(result_list), i+1)
          pyplot.axis('off')
          # plot face
          # pyplot.imshow(data[y1:y2, x1:x2])
          crop_face.append(data[y1:y2, x1:x2])
        # pyplot.show()
        return crop_face


    def get_face(self,path):
        # filename = path
        # pixels = pyplot.imread(filename)
        pixels = path
        # create the detector, using default weights
        detector = MTCNN()
        # detect faces in the image
        faces = detector.detect_faces(pixels)
        # display faces
        Face = self.mtcnn_model(pixels, faces)
        return Face , faces

    def recognition(self,):
      pass

    def save_faces(self,data_face):
      folder_name = 'Faces'+f'{self.folder_number}'
      if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
      cv2.imwrite(folder_name + '/' + f'{self.face_number}.jpg',data_face)
      self.face_number += 1

    def face_counting(self,):
      pass

In [ ]:
path_image = '/content/14605565-c91a-49b0-ba59-39c70413e734.jpeg'
path_video = '/content/video1.mp4'
output = 'C:/Users/Zahra.Dabiri/Downloads'
data_loaded = Data_Using()
face_result = Face_Detection_Recognition_Video()
data = data_loaded.load_data_video(path_video)

# cam = data_loaded.load_data_colab_camera()
face_dataset = []
for i in range(len(data)):
  faces , pose = face_result.get_face(data[i])
  face_dataset.append({'face':faces,'position':pose})
  for j in range(len(faces)):
    face_result.save_faces(faces[j])


1/1 [==============================] - 0s 158ms/step
